<h1> Python Data Notebook (Template) </h1>
<h3> Preserve this template, duplicate this file before using it!!</h3>
<strong> Author: Micah Simmerman, Nolan Ollada, Nathan Palmer </strong>

<strong>Resource URL:</strong>

<strong>Database file(s), columnar dataset(s), etc.:</strong>

In [9]:
# Libraries and packages
import os
import sqlite3
import pandas as pd
pd.__version__
import numpy as np
from scipy import stats
from collections import Counter
import math

# SQLite db file connection is achieved in the try/except statements below. 
# TODO: add sections to read-in mySQL/PostGreSQL and csv files.

db_file = '../California_Collision_Data.sqlite'  # name the db file name or resource file path.
if os.path.isfile(db_file):  # determine if there is any SQLite db file by that name exists in the specified file location.
    sqliteConnection = sqlite3.connect(db_file)  # establish a connection if the file does exist.
    cursor = sqliteConnection.cursor()  # create a cursor object.
    print('DB file and Dependencies Loaded.')  
else:
    print("No SQLite file detected.")  # otherwise, print the connection status.

DB file and Dependencies Loaded.


Start by collecting information about the table names in the database.

In [10]:
# This cell performs a search operation to collects variables used to extract data from the db file 
#  
# This cell performs the following actions:
# 1.) Extracts the name of each table in the sqlite db file. 
# 2.) Stores the attributes of each table in a 2D list by generating a list of attributes (see the inner for-loop below), 
#       creating a so called "list-of-lists". Each sublist corresponds to the same index as its' corresponding table  
#       element in the "table" list.
# Note: Each of the lists and sublists generated in this cell may be accessed by other routines in this notebook. 


# Collect some schema informatio concerning the tables in the database file.
table_query = 'SELECT name from sqlite_master where type= "table"'
tables = []  # GLOBAL TABLE INDEX
attribute_list = []  # GLOBAL ATTRIBUTE INDEX

try:
    # the first part of this algorithm collects db schema information and inserts the name of each table into a global list.
    cursor.execute(table_query)
    result = cursor.fetchall()
    print()
    print(len(result), "tables found.")
    for i in range(len(result)):
        tables.append(result[i][0])
        # print(result[i][0])

    # the second part of the algorithm extracts the attributes of each table identified in the first step and places them into a 2D list. 
    for table in tables:
        consumer_complaints_count_records = "PRAGMA table_info("
        consumer_complaints_count_records += str(table)  + ");"  # build the sqlite query string using the current list object
        cursor.execute(consumer_complaints_count_records)  # execute the query string
        result = cursor.fetchall()  # collect the results
        temp_list = []
        for item in result:
            temp_list.append(item[1])
        attribute_list.append(temp_list)  # export the list attribute table after it is built
        # print("The", table, "table contains", result[0][0], "data points")  # print the results
        # print()
    print("Table and attribute name extraction completed successfully.")
except:
    print("Error:no db connection.")


4 tables found.
Table and attribute name extraction completed successfully.


In [14]:
# Now we have a list of tables and their respective attribute columns.
print("Tables in the database: ", tables)
print("Attribute list: ", attribute_list)

Tables in the database:  ['case_ids', 'victims', 'collisions', 'parties']
Attribute list:  [['case_id', 'db_year'], ['id', 'case_id', 'party_number', 'victim_role', 'victim_sex', 'victim_age', 'victim_degree_of_injury', 'victim_seating_position', 'victim_safety_equipment_1', 'victim_safety_equipment_2', 'victim_ejected'], ['case_id', 'jurisdiction', 'reporting_district', 'population', 'county_city_location', 'county_location', 'chp_beat_type', 'primary_road', 'secondary_road', 'distance', 'direction', 'intersection', 'weather_1', 'weather_2', 'state_highway_indicator', 'caltrans_county', 'caltrans_district', 'state_route', 'route_suffix', 'postmile_prefix', 'postmile', 'location_type', 'ramp_intersection', 'side_of_highway', 'tow_away', 'collision_severity', 'killed_victims', 'injured_victims', 'party_count', 'primary_collision_factor', 'pcf_violation_code', 'pcf_violation_category', 'type_of_collision', 'motor_vehicle_involved_with', 'pedestrian_action', 'road_surface', 'road_conditio

### Database editing
run the code below to remove the attributes in the remove_cols_collisions and remove_cols_parties tables specified. You can run the above 2 code blocks again to see that the specified attributes were removed.

In [6]:
def get_new_column_string(old_columns, columns_to_remove):
    new_columns = [col for col in old_columns if col not in columns_to_remove]
    return ", ".join(new_columns)

# The columns you want to remove
remove_cols_collisions = ['officer_id', 'chp_shift', 'special_condition', 
                          'beat_type', 'city_division_lapd', 'chp_beat_class', 
                          'beat_number', 'pcf_violation', 'pcf_violation_subsection', 
                          'hit_and_run', 'process_date']
remove_cols_parties = ['hazardous_materials', 'oaf_violation_code']

# The new column strings
new_cols_collisions = get_new_column_string(attribute_list[1], remove_cols_collisions)
new_cols_parties = get_new_column_string(attribute_list[3], remove_cols_parties)

# Create new tables without the specified columns
cursor.execute(f'''
    CREATE TABLE new_collisions AS 
    SELECT {new_cols_collisions} FROM collisions WHERE 1=0;
''')

cursor.execute(f'''
    CREATE TABLE new_parties AS 
    SELECT {new_cols_parties} FROM parties WHERE 1=0;
''')

# Insert data from the old tables into the new ones
cursor.execute(f'''
    INSERT INTO new_collisions SELECT {new_cols_collisions} FROM collisions;
''')

cursor.execute(f'''
    INSERT INTO new_parties SELECT {new_cols_parties} FROM parties;
''')

# Delete the old tables
cursor.execute('DROP TABLE collisions;')
cursor.execute('DROP TABLE parties;')

# Rename the new tables to the original names
cursor.execute('ALTER TABLE new_collisions RENAME TO collisions;')
cursor.execute('ALTER TABLE new_parties RENAME TO parties;')

# Commit the changes and close the connection
sqliteConnection.commit()
sqliteConnection.close()

In [13]:
random_entries_query = """
SELECT * FROM collisions
JOIN parties ON collisions.case_id = parties.case_id
JOIN victims ON parties.case_id = victims.case_id AND parties.party_number = victims.party_number
ORDER BY RANDOM() 
LIMIT 10
"""

random_entries = pd.read_sql_query(random_entries_query, sqliteConnection)

In [6]:
# This cell will determine the number of data points within each table of the database and enumerate them one by one.

# attribute_list = []
# count_records = 'SELECT COUNT(*) FROM ('  # start the string.
# count_records += table + ");"  # concatenate the variable and complete the string build.
# cursor.execute(count_records)  # 
# result = cursor.fetchall()

for table in range(len(tables)):
    
    count_records = 'SELECT COUNT(*) FROM ('  # start the string.
    count_records += tables[table] + ");"  # concatenate the variable and complete the string build.
    cursor.execute(count_records)  # 
    result = cursor.fetchall()
    print("The", tables[table], "table contains", result[0][0], "data points")


The case_ids table contains 9424334 data points
The collisions table contains 9424334 data points
The victims table contains 9639334 data points
The parties table contains 18669166 data points


TO DO: INSERT BASIC STATISTICS FUNCTIONS BETWEEN THESE TWO CELLS.

In [7]:
# This cell will defines a first round Data Cube Function(s)
create_data_cube = """SELECT 
    c1, c2, AGGREGATE_FUNCTION(c3)
FROM
    table_name
GROUP BY CUBE(c1 , c2);"""



The functions below currently only work on csv files. Add a conditional behavior to these functions and extend them to extract equivalent data from a table in an SQLite db file. 

In [8]:
# The following are basic statistic functions that can be used as helper routines in longer extraction/cleaning/clustering efforts. 
# The remainder of the notebook contains code written to examine the telescope data. You can to generalize each function or create a
# python scrip to examine tabular data files in a terminal window.

# There are many methods that you can use to extract SQLite table data into a pandas dataframe or a numpy data object to generate graphics, 
# or perform package-based statistical testing. At this point in the notebook (and as of 11-June-2023).

import pandas as pd
import numpy as np

# maximum value (max)
def maximum(filename, attribute_number):
    if attribute_number < 1 or attribute_number > 10 or attribute_number%1!=0:
        return None
    else:
        MC_telescope_data = pd.read_csv(filename, header=None)  # read the file into a pandas dataframe
        column = MC_telescope_data.iloc[:,attribute_number]  # project the indicated column from the pandas dataframe
        result = column.max()  # find the maximum value
        return round(result, 2)

# minimum value (min)
def minimum(filename, attribute_number):
    if attribute_number < 1 or attribute_number > 10 or attribute_number%1!=0:
        return None
    else:
        MC_telescope_data = pd.read_csv(filename, header=None)
        column = MC_telescope_data.iloc[:,attribute_number]
        result = column.min()  # find the minimum value
        return round(result, 2)

# average (mean)
def mean(filename, attribute_number):
    if attribute_number < 1 or attribute_number > 10 or attribute_number%1!=0:
        return None
    else:
        MC_telescope_data = pd.read_csv(filename, header=None)
        column = MC_telescope_data.iloc[:,attribute_number]
        result = column.mean()  # calculate the average value
        return round(result, 2)

# standard deviation
def std(filename, attribute_number):
    if attribute_number < 1 or attribute_number > 10 or attribute_number%1!=0:
        return None
    else:
        MC_telescope_data = pd.read_csv(filename, header=None)
        column = MC_telescope_data.iloc[:,attribute_number]
        result = column.std()  # calculate standard deviation
        return round(result, 2)

# first quantile, Q1
def q1(filename, attribute_number):
    if attribute_number < 1 or attribute_number > 10 or attribute_number%1!=0:
        return None
    else:
        MC_telescope_data = pd.read_csv(filename, header=None)
        MC_telescope_df = pd.DataFrame(MC_telescope_data)
        column = MC_telescope_df.iloc[:,attribute_number]
        result = column.quantile([0.25]).values[0]
        return round(result, 2)

# third quantile, Q3
def q3(filename, attribute_number):
    if attribute_number < 1 or attribute_number > 10 or attribute_number%1!=0:
        return None
    else:
        MC_telescope_data = pd.read_csv(filename, header=None)
        MC_telescope_df = pd.DataFrame(MC_telescope_data)
        column = MC_telescope_df.iloc[:,attribute_number]
        result = column.quantile([0.75]).values[0]
        return round(result, 2)

# median, Q2
def median(filename, attribute_number):
    if attribute_number < 1 or attribute_number > 10 or attribute_number%1!=0:
        return None
    else:
        MC_telescope_data = pd.read_csv(filename, header=None)
        MC_telescope_df = pd.DataFrame(MC_telescope_data)
        column = MC_telescope_df.iloc[:,attribute_number]  # slice the indiced column
        result = column.quantile([0.5]).values[0]  # compute the second quantile
        return round(result, 2)

# inner-quartile range
def iqr(filename, attribute_number):
    if attribute_number < 1 or attribute_number > 10 or attribute_number%1!=0:
        return None
    else:
        iqr = float(q3(filename, attribute_number))-float(q1(filename, attribute_number))
        # return "{:.2f}".format(iqr)  # format to 2 decimal places.
        return round(iqr, 2)

# N (number of objects)
def count(filename, attribute_number):
    if attribute_number < 1 or attribute_number > 10 or attribute_number%1!=0:
        return None
    else:
        MC_telescope_data = pd.read_csv(filename, header=None)
        MC_telescope_df = pd.DataFrame(MC_telescope_data)
        column = MC_telescope_df.iloc[:,attribute_number]  # slice the indicated column
        result = column.count()  # count 
        return result

In [9]:
# OUTPUT TESTING
print("N:", count("", 7))  # N
print("Q1:", q1("", 7))  # Q1
print("median:", median("", 7))  # Median (Q2)
print("Q3:", q3("", 7))  # Q3
print("IQR:", iqr("", 7))  # IQR
print("min:", minimum("", 7))  # min
print("max:", maximum("", 7))  # max
print("average:", mean("", 7))  # mean
print("std. deviation:", std("", 7))  # standard deviation

# # ERROR HANDLING
# print(maximum("magic04.data", 2.1))  # float (in-range)
# print(minimum("magic04.data", -1))  # out of bounds (less than)
# print(mean("magic04.data", 0))  # zero
# print(std("magic04.data", -1.5))  # negative out of range float
# print(q1("magic04.data", 10.5))  # positive out of range float
# print(median("magic04.data", 11))  # positive out of range (greater than)
# print(q3("magic04.data", -11.5))  # negative out of range float (less than)

FileNotFoundError: [Errno 2] No such file or directory: ''

Next create a scatter plot using the 4th and 5th attributes. 

https://realpython.com/visualizing-python-plt-scatter/

In [10]:
import matplotlib.pyplot as plt

# attribute_names = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 'fM3Long', 'fM3Trans', 'fAlpha', 'fDist', 'class']
# Attribute Definitions
#     1.  fLength:  continuous  # major axis of ellipse [mm]
#     2.  fWidth:   continuous  # minor axis of ellipse [mm] 
#     3.  fSize:    continuous  # 10-log of sum of content of all pixels [in #phot]
#     4.  fConc:    continuous  # ratio of sum of two highest pixels over fSize  [ratio]
#     5.  fConc1:   continuous  # ratio of highest pixel over fSize  [ratio]
#     6.  fAsym:    continuous  # distance from highest pixel to center, projected onto major axis [mm]
#     7.  fM3Long:  continuous  # 3rd root of third moment along major axis  [mm] 
#     8.  fM3Trans: continuous  # 3rd root of third moment along minor axis  [mm]
#     9.  fAlpha:   continuous  # angle of major axis with vector to origin [deg]
#    10.  fDist:    continuous  # distance from origin to center of ellipse [mm]
#    11.  class:    g,h         # gamma (signal), hadron (background)

attribute_names = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 'fM3Long', 'fM3Trans', 'fAlpha', 'fDist', 'class']
MC_telescope_data = pd.read_csv("magic04.data", header=None)  # tabular file header is absent
MC_telescope_data.columns = attribute_names  # attach the column attribute labels
MC_telescope_df = pd.DataFrame(MC_telescope_data)  # create a pandas dataframe 


MC_telescope_data_column_4 = MC_telescope_df.iloc[:,4]  #  
MC_telescope_data_column_5 = MC_telescope_df.iloc[:,5]  # 

# KEEP THIS CODE FOR REFERENCE
# plt.plot(MC_telescope_data_column_4, MC_telescope_data_column_5, "o")  # use plt.plot, if you want it to be fast.
# plt.scatter(MC_telescope_data_column_4, MC_telescope_data_column_5)  # use plt.scatter, if you want more features.

plt.scatter(x=MC_telescope_data_column_4, y=MC_telescope_data_column_5)  # use plt.scatter, if you want to use more features.
plt.xlabel(attribute_names[3])  # 
plt.ylabel(attribute_names[4])  # 
plt.title("Scatter Plot of 4th and 5th Dimensions, MAGIC Telescope Data")
plt.savefig('scatter_plot_dimensions_4_5')
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [11]:
num_list = [13, 15, 16, 16, 19, 20, 20, 21, 22, 22, 25, 25, 25, 25, 30, 33, 33, 35, 35, 35, 35, 36, 40, 45, 46, 52, 70]
interval_list = {'1-5':136, '6-15':181, '16-20':178, '21-50':695, '51-80':245, '81-110':177}  # Age:Frequency


def lst_mean(lst):
    result = np.mean(lst)
    return round(result, 3)

def lst_median(lst):
    result = np.median(lst)
    return round(result, 3)

def lst_mode(lst):
    result = stats.mode(lst, keepdims=True)
    # return round(result, 3)
    return result

def lst_histogram(lst):
    a = np.array(lst)
    items = Counter(lst).keys()  # find the number of unique values in the set to set the bins
    fig, ax = plt.subplots(figsize =(5, 3))
    plt.hist(a, bins=range(np.min(lst)-5, np.max(lst)+5))

def lst_multimode(lst):
    res = []
    test_list1 = Counter(lst)
    temp = test_list1.most_common(1)[0][1]  # Extracts values of greatest frequency in the set.
    for ele in lst:
        if lst.count(ele) == temp:
            res.append(ele)
            res = list(set(res))
    # printing results
    return "Data modality = " + str(len(res)) + "; mode list = " + str(res)

def lst_midrange(lst):
    min = np.min(lst)
    max = np.max(lst)
    return round( ((min + max)/2.0), 3)

def lst_q1(lst):
    arr = np.array(lst)
    result = np.quantile(arr, 0.25)
    return round(result, 3)

def lst_q3(lst):
    arr = np.array(lst)
    result = np.quantile(arr, 0.75)
    return round(result, 3)

def cum_freq_list_from_inerval(dict):
    cumulative_freq = 0
    cumulative_freq_lst = []
    for itm in dict:
        cumulative_freq += dict[itm]  # collect the item frequency
        cumulative_freq_lst.append(cumulative_freq)
    n = cumulative_freq_lst[-1]
    return cumulative_freq_lst
    
def find_n_by_2(dict):
    cumulative_freq_lst = cum_freq_list_from_inerval(dict)
    n = cumulative_freq_lst[-1]
    n_by_2 = n/2
    return n_by_2

def find_median_bin(n_by_2, cumulative_freq_lst):
    for i in range(len(cumulative_freq_lst)):
        if n_by_2 <= cumulative_freq_lst[i]:
            return i  # returns the index of the bin the median data point belongs to. 
        
# def estimate_median(n_by_2, cumulative_freq_lst):
    

In [12]:
# num_list = [13, 15, 16, 16, 19, 20, 20, 21, 22, 22, 25, 25, 25, 25, 30, 33, 33, 35, 35, 35, 35, 36, 40, 45, 46, 52, 70]
# interval_list = {'1-5':136, '6-15':181, '16-20':178, '21-50':695, '51-80':245, '81-110':177}  # Age:Frequency

# test the functions
print("Mean: ", lst_mean(num_list))
print("Q1: ", lst_q1(num_list))
print("Q2: ", lst_median(num_list))
print("Q3: ", lst_q3(num_list))
print(lst_multimode(num_list))
print("Midrange ((min+max)/2.0) : ", lst_midrange(num_list))

# print("The data set is bimodal, with modes 25, 35.")
print(lst_histogram(num_list))  # use this to determine the mode(s) and modality of the set. 

Mean:  29.963
Q1:  20.5
Q2:  25.0
Q3:  35.0
Data modality = 2; mode list = [25, 35]
Midrange ((min+max)/2.0) :  41.5


NameError: name 'plt' is not defined

In [13]:
# Question #4 on Computing Statistics using a Frequency Distribution Table
# Estimated median = L + (L + ((n/2)-B)/G)*w
# L is the lower class boundary of the group containing the median
# n is the total number of values
# B is the cumulative frequency of the groups before the median group
# G is the frequency of the median group
# w is the group width

print("Cumm. frequency list:", cum_freq_list_from_inerval(interval_list))
print("n:", cum_freq_list_from_inerval(interval_list)[-1])
print("n/2:", find_n_by_2(interval_list))
print("Group Index:", find_median_bin(find_n_by_2(interval_list), cum_freq_list_from_inerval(interval_list)))
print("Median data point resides in age group: '21-50'")


Cumm. frequency list: [136, 317, 495, 1190, 1435, 1612]
n: 1612
n/2: 806.0
Group Index: 3
Median data point resides in age group: '21-50'
